# Part 1: Reconstruction of a positive wavefunction

## Getting Started

The following imports are needed to run this tutorial:

In [1]:
#from rbm_tutorial import RBM_Module, BinomialRBM
import torch
#from observables_tutorial import TFIMChainEnergy, TFIMChainMagnetization
import numpy as np
import csv
%matplotlib inline
import sys
sys.path.append('../qucumber/')
from positive_wavefunction import PositiveWavefunction
from quantum_reconstruction import QuantumReconstruction
import utils.observer as obs
sys.path.append('../qucumber/utils/ed/')
from hamiltonians import *
from data_generator import *
#import importlib.util
#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

*rbm_tutorial.py* contains the child class **BinomialRBM** that inherits properties and functions from the parent class **RBM_Module**. 
 
PyTorch is used as a replacement for doing some algebra that would normally be done with numpy. PyTorch also allows one to take advantage of GPU acceleration among many other things. If a GPU card is not available, the tutorial will run on a CPU by default.

*observables_tutorial.py* is a class that will allow us to calculate physical properties like the energy and magnetization from samples generated by the trained RBM.

## Training

Let's beging with training the RBM on a positive wavefunction. We consider the quantum Ising model with Hamiltonian $H=-J\sum_{\langle i j \rangle} S^z_i S^z_j - h \sum_i S^x_i$
at its quantum critical point $h/J=1$.  The training data has been generated and is contained in the file *tfim1d_N10_train_samples.txt*.  It contains 10,000 measurements of the $S^z$ states of 10 qubits, represented as zeros or ones.

To evaluate how well the RBM is training, we compute the fidelity, $|\langle \psi|\psi_{\rm RBM} \rangle|^2$, between the true wavefunction of the system and the wavefunction the RBM reconstructs. First, we need to load our training data and the true wavefunction of this system.

In [2]:
model = 'tfim1d'   #Model name
N = 10             #Number of spins
Nsamples = 1000    #Number of measurements

In [10]:
Hamiltonian=TransverseFieldIsing(N,hx=1.0)    #Build quantum Hamiltonian
(en,psi) = np.linalg.eigh(Hamiltonian)        #Full diagonalization
target_psi = psi[:,0]   #Ground state wavefunction
gs_energy = en[0]       #Ground state energy                      

In [17]:
train_samples = GenerateSamples(N,target_psi,Nsamples)

The following arguments are required to construct a **BinomialRBM** object:

1. **The number of visible units, *num_visible***. This is 10 for the case of our dataset.
2. **The number of hidden units in the hidden layer of the RBM, *num_hidden***. This number is set to the number of visible units by default (10 in the case of our dataset).

In [18]:
nv = N
nh  = nv

A **BinomialRBM** object has a function called *fit* that performs the training. *fit* takes the following arguments:

1. **train_set**: needed for selecting mini batches of the data
2. **true_psi**: only needed here to compute the fidelity
3. **epochs**: the number of epochs, i.e. training cycles that will be performed; 1000 should be fine
4. **batch_size**: the number of data points that each mini batch will contain; we'll go with 100
5. **k**: the number of contrastive divergence steps; k=1 seems to be good enough in most cases
6. **lr**: the learning rate; we will use a learning rate of 0.01 here
7. **log_every**: how often you would like the program to update you during the training; we choose 50 - that is, every 50 epochs the program will print out the fidelity

In [19]:
epochs     = 1000
num_chains = 100
batch_size = 100
k          = 10
lr         = 0.1
log_every  = 10

In [20]:
nn_state = PositiveWavefunction(num_visible=nv,num_hidden=nh, seed=1234)
qr = QuantumReconstruction(nn_state)
input_psi = torch.tensor(np.asarray([psi[:,0],np.zeros((psi.shape[0]))]),dtype=torch.double)

In [21]:
qr.fit(train_samples, epochs, batch_size, num_chains, k,lr, progbar=False,target_psi=input_psi)

Epoch = 0   Fidelity = 0.4537040747553997
Epoch = 100   Fidelity = 0.9066977717498028
Epoch = 200   Fidelity = 0.9533327102523875
Epoch = 300   Fidelity = 0.9695376282081614


KeyboardInterrupt: 

## After Training 

After training your RBM, the *fit* function will have saved your trained weights and biases. Now, we can generate samples from our trained RBM and calculate physical observables. Let's calculate the energy and the magnetization of newly generated samples.

**TFIMChainEnergy** and **TFIMChainMagnetization** objects (from *observables_tutorial.py*) have a sampler function called *sample* built into them. They will generate samples from the RBM distribution that was just learned in the training procedure, compute the observables and plot the computed values as a function of the Gibbs step.

All that needs to be done is to feed in the following arguments into the objects' *sample* functions:

1. **sampler**: the RBM object that we've trained
2. **num_samples**: the number of samples we wish to generate
3. **k**: the number of Gibbs steps that will be used to generate samples


In [ ]:
#rbm_energy = TFIMChainEnergy(h=1.0) # TFIMChainEnergy requires the h value for initialization
#energies = rbm_energy.sample(sampler=rbm_real, num_samples=3000, k=100)

In [ ]:
#rbm_mag    = TFIMChainMagnetization()
#magnetizations = rbm_mag.sample(sampler=rbm_real, num_samples=3000, k=100)

You can see a brief transient period in each observable, before the state of the machine "warms up" to equilibrium.  After that, the values fluctuate around the mean.  The exact value for the energy is -1.2785, and for the magnetization is 0.7072.

And there you have it! For more information on using QuCumber on your machine, please refer to [here](../tutorial.rst). If you are interested in learning about training a **ComplexRBM** object, please click next.